## 1. Retrieve




### Import packages

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
##This is so variables get printed within jupyter
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# ##import packages
# import cdsapi ## check the current working directory, which should be the UNSEEN-open directory
# import numpy as np
# import os
# import xarray as xr
# import matplotlib.pyplot as plt
# import cartopy
# import cartopy.crs as ccrs

In [4]:
import sys
sys.path.append('../../')

In [5]:
import src.CDSretrieve as retrieve

In [6]:
# ##We want the working directory to be the UNSEEN-open directory
# pwd = os.getcwd() ##current working directory is UNSEEN-open/Notebooks/1.Download
# pwd #print the present working directory
# os.chdir(pwd+'/../../') # Change the working directory to UNSEEN-open
# os.getcwd() #print the working directory

## Download all data

We will be using the SEAS5 hindcast, which is a dataset running from 1981-2016. The hindcast is initialized every month with 25 ensemble members and the forecast run for 6 months, indicated by blue horizontal bars below. February is forecasted by 6 initialization months (September-February). We discard the first month of the forecast because of dependence between the forecasts, explained in the evaluation section and are left with 5 initialization months (Sep-Jan) and 25 ensemble members forecasting february precipitation each year, totalling to an increase of 125 times the observed length.

For a summary of all available C3S seasonal hindcasts, their initialization months and more specifics, please see [ECMWF page](https://confluence.ecmwf.int/display/CKB/C3S+Seasonal+Forecasts) and the [SEAS5 paper](https://doi.org/10.5194/gmd-12-1087-2019).  



![title](../../../graphs/Data_illustration.png)

The first download example above downloaded all 25 ensemble members for the forecast initialized in January (the bottom bar). We should repeat this over the other initialization month and over all years (1981-2016).

In [7]:
init_months, leadtimes = retrieve._get_init_months(target_month=[3,5,6])
init_months
leadtimes

[2, 1, 12]

[array([2, 3, 4]), array([3, 4, 5]), array([4, 5, 6])]

Leadtimes should be [[2,3,4],[3,4,5],[4,5,6]]

In [10]:
retrieve.print_arguments([3,4,5])

year = 1982 init_month = 2 leadtime_month = [2 3 4]
year = 1982 init_month = 1 leadtime_month = [3 4 5]
year = 1981 init_month = 12 leadtime_month = [4 5 6]


In [13]:
retrieve.print_arguments([6,7,8])

year = 1981 init_month = 5 leadtime_month = [2 3 4]
year = 1981 init_month = 4 leadtime_month = [3 4 5]
year = 1981 init_month = 3 leadtime_month = [4 5 6]
year = 1982 init_month = 5 leadtime_month = [2 3 4]
year = 1982 init_month = 4 leadtime_month = [3 4 5]
year = 1982 init_month = 3 leadtime_month = [4 5 6]


In [13]:
retrieve.print_arguments([3,4,5], years=range(2016,2021))

year = 2017 init_month = 2 leadtime_month = [2 3 4]
year = 2017 init_month = 1 leadtime_month = [3 4 5]
year = 2016 init_month = 12 leadtime_month = [4 5 6]
year = 2018 init_month = 2 leadtime_month = [2 3 4]
year = 2018 init_month = 1 leadtime_month = [3 4 5]
year = 2017 init_month = 12 leadtime_month = [4 5 6]
year = 2019 init_month = 2 leadtime_month = [2 3 4]
year = 2019 init_month = 1 leadtime_month = [3 4 5]
year = 2018 init_month = 12 leadtime_month = [4 5 6]
year = 2020 init_month = 2 leadtime_month = [2 3 4]
year = 2020 init_month = 1 leadtime_month = [3 4 5]
year = 2019 init_month = 12 leadtime_month = [4 5 6]


In [10]:
retrieve.print_arguments(2)

year = 1982 init_month = 1 leadtime_month = [2]
year = 1981 init_month = 12 leadtime_month = [3]
year = 1981 init_month = 11 leadtime_month = [4]
year = 1981 init_month = 10 leadtime_month = [5]
year = 1981 init_month = 9 leadtime_month = [6]


In [11]:
retrieve.print_arguments(2, years=[2015,2016])

year = 2016 init_month = 1 leadtime_month = [2]
year = 2015 init_month = 12 leadtime_month = [3]
year = 2015 init_month = 11 leadtime_month = [4]
year = 2015 init_month = 10 leadtime_month = [5]
year = 2015 init_month = 9 leadtime_month = [6]


For our download, we loop over initialization months and years. Because we only want February precipitation, the leadtime month (forecast length) changes with the initialization month. For example, in the September initialized forecasts, we only want the leadtime month 6 = February. For August initialized this is leadtime 5, etc. Furthermore, the year the forecast is initialized is required for the download. For September - December initialized forecasts, this is the target year-1. For January it is the same year as the target year. For example, for the first two target years this looks like the following:

Write a function that is used for the download.

And start the download! In total, we request 35 years x  initialization dates = 175 requests. 
I could try sending just 5 request of the different initialization dates for all years?

In [13]:
os.getcwd() ##current working directory is UNSEEN-open/Notebooks/1.Download

'/lustre/soge1/projects/ls/personal/timo/UNSEEN-open'

In [34]:
retrieve_SEAS5(variables = ['2m_temperature','2m_dewpoint_temperature'], target_months = [6,7,8], 
         area = [70, -11, 30, 120], folder = '../../../Siberia_example/')

In [34]:
retrieve_SEAS5(variables = 'total_precipitation', target_months = [2], 
         area = [60, -11, 50, 2], folder = '../UK_example/')

## ERA5 data download

ERA5 is downloaded using the monthly means for single levels `reanalysis-era5-single-levels-monthly-means`. 
The function takes the variables, target months, area and folder. The function downloads the data on a 1x1 degree grid, similar to the SEAS5 data. The default dataset is 0.25 x 0.25 (so higher resolution data is available). 


In [2]:
def retrieve_ERA5(variables,
                  folder,
                  grid = [1.0, 1.0],
                  target_months=[
                      '01',
                      '02',
                      '03',
                      '04',
                      '05',
                      '06',
                      '07',
                      '08',
                      '09',
                      '10',
                      '11',
                      '12',
                  ],
                  area=[90, -180, -90, 180]):
    """Retrieves the full ERA5 dataset from CDS (years 1979-2020).
        
        Parameters
        ----------
        variables : The variables to be downloaded, str. Can also be one variable.
        folder : The path to the folder where to store the data. 
        grid : The grid spacing, or spatial resolution of the data. 
            Defaults to 1x1 degrees to match SEAS5.
            If a higher resolution is wanted, use [0.25, 0.25].
        target_months : The month(s) of interest.
            For example, for JJA, use [6,7,8]. 
            Defaults to all months. 
        area : The domain to download the data over, [North, West, South, East,].
            For example, to dowload longitude 30,70 and latitude -10, 120, use [70, -11, 30, 120,].
            Default is the full extent [90, -180, -90, 180].

        Returns
        -------
        Saves the files in the specified folder.
    """
    c.retrieve(
        'reanalysis-era5-single-levels-monthly-means',
        {
            'format':
            'netcdf',
            'product_type':
            'monthly_averaged_reanalysis',
            'variable': variables,
            'area':     area,         
            'grid':     [1.0, 1.0],  
            'year': [
                '1979',
                '1980',
                '1981',
                '1982',
                '1983',
                '1984',
                '1985',
                '1986',
                '1987',
                '1988',
                '1989',
                '1990',
                '1991',
                '1992',
                '1993',
                '1994',
                '1995',
                '1996',
                '1997',
                '1998',
                '1999',
                '2000',
                '2001',
                '2002',
                '2003',
                '2004',
                '2005',
                '2006',
                '2007',
                '2008',
                '2009',
                '2010',
                '2011',
                '2012',
                '2013',
                '2014',
                '2015',
                '2016',
                '2017',
                '2018',
                '2019',
                '2020',
            ],
            'month':
            target_months,
            'time':
            '00:00',
        },
        folder + 'ERA5_1x1.nc')

In [ ]:
def retrieve_SEAS5(variables, target_months, area, folder, years=np.arange(1981, 2017)):
    """Retrieve SEAS5 data from CDS.
        
        Parameters
        ----------
        variables : The variables to be downloaded, str. Can also be one variable.
        target_months : The month(s) of interest, int.
            For example, for JJA, use [6,7,8]. 
            Must be consecutive (e.g. no May, July, August).
        area : The domain to download the data over, [North, West, South, East,].
            For example, to dowload longitude 30,70 and latitude -10, 120, use [70, -11, 30, 120,]
        years : Defaults to the entire hindcast years from 1981-2016
        folder : The path to the folder where to store the data. 

        Returns
        -------
        Saves the files in the specified folder.
    """
    init_months, leadtimes = _get_init_months(target_months)
    init_months_str = ''.join(str(a) for a in init_months)
    if '112' in init_months_str: ## This means that we are crossing years - both January and December initializations are used. 
        for j in range(len(years)-1): 
            for i in range(len(init_months)):
                init_month = init_months[i]
                leadtime_months = leadtimes[i]
                if 12 in init_months:
                    if init_month < 6:
                        year = years[j] + 1
                    else:
                        year = years[j]
                else:
                    year = years[j]

                if not os.path.isfile(folder + str(year) + "%.2i" % init_month + '.nc'):
                    _retrieve_single(variables=variables,
                                    year=year,
                                    init_month=init_month,
                                    leadtimes=leadtime_months,
                                    area = area,
                                    folder=folder)